In [1]:
import pandas as pd
import numpy as np

In [10]:
data = pd.read_csv(r"C:\Users\wasif\Downloads\ml-latest-small\ml-latest-small\ratings.csv")
print(data.shape)
data = data.drop(columns=['timestamp'])
data.head(5)

(100836, 4)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [11]:
movie_titles = pd.read_csv(r"C:\Users\wasif\Downloads\ml-latest-small\ml-latest-small\movies.csv")
movie_titles = movie_titles.drop(columns=['genres'])
movie_titles.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [12]:
data = pd.merge(data, movie_titles, on = 'movieId')
data.head(500)

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
495,369,47,3.5,Seven (a.k.a. Se7en) (1995)
496,370,47,3.5,Seven (a.k.a. Se7en) (1995)
497,372,47,5.0,Seven (a.k.a. Se7en) (1995)
498,373,47,5.0,Seven (a.k.a. Se7en) (1995)


In [13]:
moviemat = data.pivot_table(index ='userId', columns = 'title', values = 'rating')

In [44]:
moviemat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


In [14]:
def pearson_correlation(user1, user2): 
    # Extract the rows corresponding to the two users
    user1 = moviemat.loc[user1]
    user2 = moviemat.loc[user2]
    
    # Find the common movies that both users have rated (non-missing values)
    common_items = np.logical_and(~np.isnan(user1), ~np.isnan(user2))
    
    # If there are no common movies, return NaN to indicate no correlation
    if not common_items.any():
        return np.nan
    
    # Calculate the Pearson correlation for the common items
    user1_common = user1[common_items]
    user2_common = user2[common_items]

    # Calculate the means of the common ratings
    mean_user1 = np.nanmean(user1_common)
    mean_user2 = np.nanmean(user2_common)

    # Calculate the numerator and denominators for the Pearson correlation formula
    numerator = np.sum((user1_common - mean_user1) * (user2_common - mean_user2))
    denominator_user1 = np.sqrt(np.sum((user1_common - mean_user1) ** 2))
    denominator_user2 = np.sqrt(np.sum((user2_common - mean_user2) ** 2))
    if(denominator_user1*denominator_user2==0):
        return np.nan

    # Calculate the Pearson correlation coefficient
    correlation = numerator / (denominator_user1 * denominator_user2)
    
    return correlation

In [15]:
def top_similar_users(user, all_users, moviemat, num_users=10):
    # Calculate Pearson correlation between the given user and all other users
    correlations = []
    for other_user in all_users:
        if user != other_user:
            correlation = pearson_correlation(user, other_user)
            if not np.isnan(correlation):
                correlations.append((other_user, correlation))

    # Sort the correlations in descending order
    correlations.sort(key=lambda x: x[1], reverse=True)

    # Select the top num_users similar users
    top_similar_users = correlations[:num_users]

    return top_similar_users

# Example usage:
# Replace 'user' with the user you want to find similar users for and 'all_users' with the list of all users in your dataset.
# user = 1 # Replace with the actual user name or ID
# all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
# top_2_similar_users = top_similar_users(user, all_users, moviemat, num_users=2)

# print("Top 2 most similar users to", user, "are:")
# for similar_user, correlation in top_2_similar_users:
#     print(similar_user, "with correlation", correlation)

In [17]:
user = 1 # Replace with the actual user name or ID
all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
top_50_similar_users_1 = top_similar_users(user, all_users, moviemat, num_users=10)

print("Top 10 most similar users to", user, "are:")
for similar_user, correlation in top_50_similar_users_1:
    print(similar_user, "with correlation", correlation)

Top 10 most similar users to 1 are:
146 with correlation 1.0000000000000002
550 with correlation 1.0000000000000002
106 with correlation 1.0
333 with correlation 1.0
598 with correlation 0.9999999999999998
473 with correlation 0.9622504486493763
511 with correlation 0.9258200997725514
9 with correlation 0.9185586535436917
13 with correlation 0.8783100656536798
366 with correlation 0.8728715609439696


In [18]:
# Extract the user IDs from the top 10 similar users list
similar_user_ids_1 = [user_id for user_id, _ in top_50_similar_users_1]

# Include user=1 in the list of similar users
similar_user_ids_1.append(user)

# Create a new DataFrame with only the selected users
new_matrix_user1 = moviemat.loc[similar_user_ids_1]

In [19]:
new_matrix_user1

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#For 2nd user=146
user = 146 # Replace with the actual user name or ID
all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
top_10_similar_users_2 = top_similar_users(user, all_users, moviemat, num_users=10)

print("Top 10 most similar users to", user, "are:")
for similar_user, correlation in top_10_similar_users_2:
    print(similar_user, "with correlation", correlation)

Top 10 most similar users to 146 are:
1 with correlation 1.0000000000000002
135 with correlation 1.0000000000000002
149 with correlation 1.0000000000000002
221 with correlation 1.0000000000000002
428 with correlation 1.0000000000000002
464 with correlation 1.0000000000000002
512 with correlation 1.0000000000000002
10 with correlation 1.0
14 with correlation 1.0
46 with correlation 1.0


In [21]:
#For 2nd user=146
user=146
# Extract the user IDs from the top 10 similar users list
similar_user_ids_2 = [user_id for user_id, _ in top_10_similar_users_2]

# Include user=1 in the list of similar users
similar_user_ids_2.append(user)

# Create a new DataFrame with only the selected users
new_matrix_user2 = moviemat.loc[similar_user_ids_2]


In [22]:
#For 3rd user=550
user = 550 # Replace with the actual user name or ID
all_users = moviemat.index  # Get the list of user IDs from the index of moviemat
top_10_similar_users_3 = top_similar_users(user, all_users, moviemat, num_users=10)

print("Top 10 most similar users to", user, "are:")
for similar_user, correlation in top_10_similar_users_3:
    print(similar_user, "with correlation", correlation)

Top 10 most similar users to 550 are:
1 with correlation 1.0000000000000002
69 with correlation 1.0000000000000002
72 with correlation 1.0000000000000002
121 with correlation 1.0000000000000002
135 with correlation 1.0000000000000002
335 with correlation 1.0000000000000002
385 with correlation 1.0000000000000002
428 with correlation 1.0000000000000002
430 with correlation 1.0000000000000002
572 with correlation 1.0000000000000002


In [23]:
#For 3rd user=550
user=550
# Extract the user IDs from the top 10 similar users list
similar_user_ids_3 = [user_id for user_id, _ in top_10_similar_users_3]

# Include user=1 in the list of similar users
similar_user_ids_3.append(user)

# Create a new DataFrame with only the selected users
new_matrix_user3 = moviemat.loc[similar_user_ids_3]


In [27]:
#New code
# Function for predicting rating of a movie
def predict_rating(item, similarity_scores, moviemat):
    # Find the users who have rated the item and have non-NaN values
    item = moviemat.columns[item]
    rated_users = ~np.isnan(moviemat[item])
    user = 1  # Specify the user for whom you want to make predictions
    
    # Get the similarity scores
    similarity = similarity_scores
    
    # Initialize the weighted sum and sum of similarity scores
    weighted_sum = 0
    total_similarity = 0
    
    # Iterate through users who have rated the item
    for other_user in moviemat.index:
        if other_user != user and rated_users[other_user]:
            # Calculate the weighted contribution to the prediction
            similarity_value = similarity.get(other_user, 0)  # Use get() to handle missing keys
            other_user_rating = moviemat.at[other_user, item]
            weighted_sum += similarity_value * (other_user_rating - np.nanmean(moviemat.loc[other_user]))
            total_similarity += np.abs(similarity_value)
    
    if total_similarity == 0:
        # Handle the case where there are no similar users
        return np.nan
    
    # Calculate the predicted rating by adding the user's mean rating
    # and the weighted sum of ratings from similar users
    user_ratings = moviemat.loc[user]
    user_mean = np.nanmean(user_ratings)
    predicted_rating = user_mean + (weighted_sum / total_similarity)
    predicted_rating = min(5, max(0, predicted_rating))
    return predicted_rating

# Example usage:
# predicted_rating = predict_rating(item, similarity, new_matrix)


In [31]:
user_1_mat = new_matrix_user1.iloc[10:11].copy()
user_1_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN


In [33]:
# New code
# Find the movies that User 1 hasn't rated
user = 1
user1_ratings = new_matrix_user1.loc[user]
unrated_movies = user1_ratings.isna()

# Initialize a list to store predicted ratings, movie indices, and their corresponding movie titles
predicted_ratings = []

# Iterate through unrated movies and make predictions
for item in range(len(new_matrix_user1.columns)):
    if unrated_movies[item]:
        predicted_rating = predict_rating(item, similarity, moviemat)
        # if not np.isnan(predicted_rating):
        #    print(f"{new_matrix_user1.columns[item]}: {predicted_rating}")

        # Update user_1_mat directly with the predicted rating
        column_name = new_matrix_user1.columns[item]
        user_1_mat.loc[user, column_name] = predicted_rating
        predicted_ratings.append((column_name, predicted_rating))

# Sort the predicted ratings in descending order
predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 rated movies that User 1 hasn't rated yet
#top_10_unrated_movies = predicted_ratings[:10]

# Extract the movie titles and ratings based on their indices
top_10_movie_info = [(new_matrix.columns[item], rating) for item, rating in top_10_unrated_movies]

# Print the top 10 rated movies that User 1 hasn't rated yet with their ratings
print("Top 10 rated movies that User 1 hasn't rated yet:")
for movie_title, rating in top_10_movie_info:
    print(f"{movie_title}: {rating}")


C:\Users\wasif\AppData\Local\Temp\ipykernel_11432\2939830986.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if unrated_movies[item]:


Top 10 rated movies that User 1 hasn't rated yet:


NameError: name 'top_10_movie_info' is not defined

In [34]:
#For user 2

user_2_mat = new_matrix_user2.iloc[10:11].copy()
user_2_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#For User 2
# New code
# Find the movies that User 1 hasn't rated
user = 146
user2_ratings = new_matrix_user2.loc[user]
unrated_movies = user2_ratings.isna()

# Initialize a list to store predicted ratings, movie indices, and their corresponding movie titles
predicted_ratings = []

# Iterate through unrated movies and make predictions
for item in range(len(new_matrix_user2.columns)):
    if unrated_movies[item]:
        predicted_rating = predict_rating(item, similarity2, moviemat)
        # if not np.isnan(predicted_rating):
        #    print(f"{new_matrix_user1.columns[item]}: {predicted_rating}")

        # Update user_1_mat directly with the predicted rating
        column_name = new_matrix_user2.columns[item]
        user_2_mat.loc[user, column_name] = predicted_rating
        predicted_ratings.append((column_name, predicted_rating))

# Sort the predicted ratings in descending order
#predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 rated movies that User 1 hasn't rated yet
#top_10_unrated_movies = predicted_ratings[:10]

# Extract the movie titles and ratings based on their indices
#top_10_movie_info = [(new_matrix.columns[item], rating) for item, rating in top_10_unrated_movies]

# Print the top 10 rated movies that User 1 hasn't rated yet with their ratings
# print("Top 10 rated movies that User 1 hasn't rated yet:")
# for movie_title, rating in top_10_movie_info:
#     print(f"{movie_title}: {rating}")


C:\Users\wasif\AppData\Local\Temp\ipykernel_11432\565422314.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if unrated_movies[item]:


In [ ]:
#For user 3

user_3_mat = new_matrix_user3.iloc[10:11].copy()
user_3_mat

In [36]:
#For user 2
# Assuming 'user' is the user for whom you want to find similar users
user = 146

# Get the list of all users
all_users = moviemat.index

# Initialize a dictionary to store similarities
similarity2 = {}

# Calculate Pearson correlation for each user using your custom function
for other_user in all_users:
    if user != other_user:
        correlation = pearson_correlation(user, other_user)
        
        if not np.isnan(correlation):
            similarity2[other_user] = correlation
        else:
            print(f"No common movies between user {user} and user {other_user}")

# Example usage of similarity dictionary
item = 146  # Replace with the actual item you want to predict
predicted_rating = predict_rating(item, similarity, new_matrix)
print(f"Predicted rating for item {item}: {predicted_rating}")



No common movies between user 146 and user 2
No common movies between user 146 and user 3
No common movies between user 146 and user 4
No common movies between user 146 and user 5
No common movies between user 146 and user 7
No common movies between user 146 and user 8
No common movies between user 146 and user 9
No common movies between user 146 and user 11
No common movies between user 146 and user 12
No common movies between user 146 and user 13
No common movies between user 146 and user 15
No common movies between user 146 and user 16
No common movies between user 146 and user 17
No common movies between user 146 and user 21
No common movies between user 146 and user 22
No common movies between user 146 and user 23
No common movies between user 146 and user 25
No common movies between user 146 and user 26
No common movies between user 146 and user 27
No common movies between user 146 and user 29
No common movies between user 146 and user 30
No common movies between user 146 and use

NameError: name 'new_matrix' is not defined

In [38]:
#For user 3
# Assuming 'user' is the user for whom you want to find similar users
user = 550

# Get the list of all users
all_users = moviemat.index

# Initialize a dictionary to store similarities
similarity3 = {}

# Calculate Pearson correlation for each user using your custom function
for other_user in all_users:
    if user != other_user:
        correlation = pearson_correlation(user, other_user)
        
        if not np.isnan(correlation):
            similarity3[other_user] = correlation
        else:
            print(f"No common movies between user {user} and user {other_user}")

# Example usage of similarity dictionary
item = 146  # Replace with the actual item you want to predict
predicted_rating = predict_rating(item, similarity, new_matrix)
print(f"Predicted rating for item {item}: {predicted_rating}")




No common movies between user 550 and user 3
No common movies between user 550 and user 4
No common movies between user 550 and user 6
No common movies between user 550 and user 7
No common movies between user 550 and user 9
No common movies between user 550 and user 12
No common movies between user 550 and user 13
No common movies between user 550 and user 20
No common movies between user 550 and user 23
No common movies between user 550 and user 26
No common movies between user 550 and user 27
No common movies between user 550 and user 29
No common movies between user 550 and user 30
No common movies between user 550 and user 31
No common movies between user 550 and user 35
No common movies between user 550 and user 36
No common movies between user 550 and user 37
No common movies between user 550 and user 39
No common movies between user 550 and user 40
No common movies between user 550 and user 44
No common movies between user 550 and user 48
No common movies between user 550 and u

NameError: name 'new_matrix' is not defined

In [39]:
#For user 3

user_3_mat = new_matrix_user3.iloc[10:11].copy()
user_3_mat

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
#For User 3
# New code
# Find the movies that User 1 hasn't rated
user = 550
user3_ratings = new_matrix_user3.loc[user]
unrated_movies = user3_ratings.isna()

# Initialize a list to store predicted ratings, movie indices, and their corresponding movie titles
predicted_ratings = []

# Iterate through unrated movies and make predictions
for item in range(len(new_matrix_user3.columns)):
    if unrated_movies[item]:
        predicted_rating = predict_rating(item, similarity3, moviemat)
        # if not np.isnan(predicted_rating):
        #    print(f"{new_matrix_user1.columns[item]}: {predicted_rating}")

        # Update user_1_mat directly with the predicted rating
        column_name = new_matrix_user3.columns[item]
        user_3_mat.loc[user, column_name] = predicted_rating
        predicted_ratings.append((column_name, predicted_rating))

# Sort the predicted ratings in descending order
#predicted_ratings.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 rated movies that User 1 hasn't rated yet
#top_10_unrated_movies = predicted_ratings[:10]

# Extract the movie titles and ratings based on their indices
#top_10_movie_info = [(new_matrix.columns[item], rating) for item, rating in top_10_unrated_movies]

# Print the top 10 rated movies that User 1 hasn't rated yet with their ratings
# print("Top 10 rated movies that User 1 hasn't rated yet:")
# for movie_title, rating in top_10_movie_info:
#     print(f"{movie_title}: {rating}")


C:\Users\wasif\AppData\Local\Temp\ipykernel_11432\472173497.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if unrated_movies[item]:


In [41]:
merged_df = pd.concat([user_1_mat, user_2_mat, user_3_mat], axis=0)
merged_df

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.054935,4.767128,4.558392,3.269605,3.574265,2.605657,4.310777,3.967423,4.661113,4.449123,...,3.019008,4.861062,4.556629,4.230219,NaN,4.636781,3.442791,3.720231,4.000000,5.0
146,4.677823,4.767128,4.016885,5.000000,5.000000,NaN,3.785561,3.967423,4.697070,4.508600,...,3.019008,4.821116,4.041756,3.682124,3.661155,4.747605,3.669275,2.764814,3.851098,NaN
550,4.054935,3.965630,4.465630,NaN,NaN,2.605657,3.988502,3.967423,4.616990,4.496454,...,3.019008,4.461231,4.802226,4.566842,3.661155,4.659989,4.145344,3.483887,3.726868,NaN


In [42]:
#merged_df
merged_df_new = merged_df.dropna(axis=1)
merged_df_new


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),00 Schneider - Jagd auf Nihil Baxter (1994),10 (1979),...,Zootopia (2016),Zulu (1964),Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.054935,4.767128,4.558392,4.310777,3.967423,4.661113,4.449123,4.388685,5.0,4.480970,...,4.365022,5.000000,3.019008,4.861062,4.556629,4.230219,4.636781,3.442791,3.720231,4.000000
146,4.677823,4.767128,4.016885,3.785561,3.967423,4.697070,4.508600,4.182624,5.0,3.889919,...,4.624343,4.163011,3.019008,4.821116,4.041756,3.682124,4.747605,3.669275,2.764814,3.851098
550,4.054935,3.965630,4.465630,3.988502,3.967423,4.616990,4.496454,3.625136,5.0,4.499789,...,4.392819,4.036652,3.019008,4.461231,4.802226,4.566842,4.659989,4.145344,3.483887,3.726868


In [43]:
# Calculate the mean rating for each movie
mean_ratings = merged_df_new.mean()

# Sort the movies by mean rating in descending order
sorted_mean_ratings = mean_ratings.sort_values(ascending=False)

# Print the top 10 highest-rated movies
top_10_movies = sorted_mean_ratings.head(10)

# Print the results
print("Top 10 highest-rated movies following Average Method:")
for title, rating in top_10_movies.items():
    print(f"{title}: {rating}")


Top 10 highest-rated movies following Average Method:
Lammbock (2001): 5.0
George Carlin: It's Bad for Ya! (2008): 5.0
George Carlin: Life Is Worth Losing (2005): 5.0
George Carlin: You Are All Diseased (1999): 5.0
Real Women Have Curves (2002): 5.0
Wallace & Gromit: The Best of Aardman Animation (1996): 5.0
Ghost in the Shell 2.0 (2008): 5.0
Ran (1985): 5.0
Ghost in the Shell: Solid State Society (2006): 5.0
Burke and Hare (2010): 5.0


In [52]:
import pandas as pd

# Assuming merged_df is your DataFrame
# merged_df = ...

# Find NaN values in merged_df
nan_values = merged_df.isna()

# Display columns with NaN values
columns_with_nan = nan_values.all()
columns_with_nan = columns_with_nan[columns_with_nan].index.tolist()
print("Columns with NaN values:", columns_with_nan)


Columns with NaN values: ['Alvarez Kelly (1966)', 'Broken English (1996)', 'Cinderella (1997)', 'Circus (2000)', 'Cold Creek Manor (2003)', 'Come See the Paradise (1990)', 'Devil and Daniel Johnston, The (2005)', 'Entertaining Angels: The Dorothy Day Story (1996)', 'Harlan County U.S.A. (1976)', 'Jungle Book 2, The (2003)', "Jupiter's Wife (1994)", 'Little Ashes (2008)', 'My Blueberry Nights (2007)', 'Phantom Tollbooth, The (1970)', 'Raise Your Voice (2004)', 'True Stories (1986)', 'War Dance (2007)']


In [49]:
sum_of_ratings = merged_df["'Salem's Lot (2004)"].sum()

print("Sum of values in the 'rating' column:", sum_of_ratings)

Sum of values in the 'rating' column: 8.269605116796441


In [53]:
# Print the values of the "rating" column
rating_column_values = merged_df["Alvarez Kelly (1966)"]
print("Values of the 'rating' column:")
print(rating_column_values)

Values of the 'rating' column:
userId
1     NaN
146   NaN
550   NaN
Name: Alvarez Kelly (1966), dtype: float64


In [61]:
def explanation(column_name):
    # Assuming merged_df is your DataFrame
    # merged_df = ...

    # Check if the column has any NaN values
    if merged_df[column_name].isna().all():
        print(f"No ratings available for the '{column_name}' column.")
    else:
        # Check if the average is below 2.7
        average_rating = merged_df[column_name].mean()
        if pd.isna(average_rating):
            print(f"No ratings available for the '{column_name}' column.")
        elif average_rating < 2.7:
            print(f"The average rating for the '{column_name}' column is below 2.7. It can't be recommended.")
        else:
            print(f"The average rating for the '{column_name}' column is acceptable but doesn't make it to the top list.")

# Example usage:
explanation("Broken English (1996)")
explanation("'Tis the Season for Love (2015)")
explanation("'Hellboy': The Seeds of Creation (2004)")


No ratings available for the 'Broken English (1996)' column.
The average rating for the ''Tis the Season for Love (2015)' column is below 2.7. It can't be recommended.
The average rating for the ''Hellboy': The Seeds of Creation (2004)' column is acceptable but doesn't make it to the top list.
